# Understanding the performance of the vecadd kernel

_Vasileios Karakasis, CSCS_

In this section we will see how we can avoid the JIT cost of Numba, how we can measure the performance of the kernel without the `%timeit` magic, how we can use `nvprof`, the CUDA profiler to analyze the performance of the kernel, and finally, we will evaluate the performance of the kernel.

## Avoiding the JIT cost

The previous exercise has shown that Numba will compile the CUDA kernel every time we call our program and, in order to amortize the compilation cost, we need several invocations. We would like to avoid this cost.

Unlike the `@numba.jit` decorator, `@cuda.jit` does not accept a `cache` parameter, that would cache the generated code on the disk and use it on subsequent invocations of the program. Nonetheless, we can force the code generation at import time by supplying a function signature to the `@cuda.jit` decorator that describes the CUDA kernel. This will generate the CUDA code at the time when the decorator processes the function declaration and, therefore, we will avoid the runtime cost of JIT. Let's see how this is done:

In [1]:
import numba.cuda as cuda
import numpy as np


@cuda.jit('void(Array(float64, 1, "C"), Array(float64, 1, "C"), Array(float64, 1, "C"))')
def _vecadd_cuda(z, x, y):
    '''The CUDA kernel'''
    i = cuda.grid(1)
    N = x.shape[0]
    if i >= N:
        return

    z[i] = x[i] + y[i]


This instructs the Numba runtime to compile the following function into a CUDA kernel (return type `void`) accepting three one-dimensional arrays of `float64` (or `double`) stored in row-major order (C convention). This way, Numba does not have to wait until the `_vecadd_cuda` function is called to figure out the argument types and compile the kernel. It can do this at import time, when it first encounters the function. The downside to that is that you can't call the function with a different type of arguments later. For more details on how you can specify function signatures in Numba, see [here](http://numba.pydata.org/numba-doc/latest/reference/types.html#numba-types).

Let's retry our example now with this version of the kernel.

In [4]:
# Set up the host vectors
N = 200*1000
x = np.random.rand(N)
y = np.random.rand(N)

# Copy and allocate data on the device
d_x = cuda.to_device(x)
d_y = cuda.to_device(y)
d_z = cuda.device_array_like(x)

# Set up the kernel invocation
block_size = 128
num_blocks = N // block_size
if N % block_size:
    num_blocks += 1

# Call the kernel
_vecadd_cuda[num_blocks, block_size](d_z, d_x, d_y)

# Copy back the result to the host
res = d_z.copy_to_host()

# Validate the result
assert np.allclose(x + y, res)

### Exercise

> Time the kernel with `%timeit -n1 -r1`. Try to increase the repetitions and experiment with different array sizes. What do you see?

## Measuring the execution time of the kernel

All you see from the previous exercise is the same execution time! What is happening? Actually, you are not measuring the kernel execution time, but rather the kernel launch time. CUDA kernels are launched asynchronously. This means that as soon as you launch the kernel on the GPU, the CPU will continue execution. In this case, it will continue executing and it will block at the statement that copies back the result to the host. 

How do we measure the kernel execution time then? For this, we are going to write a Python [context manager](https://docs.python.org/3.8/reference/datamodel.html?highlight=__getitem__#with-statement-context-managers) so as to measure the execution time of a region in a nice, Pythonic way:

In [4]:
import time


class time_region:
    def __init__(self, time_offset=0):
        self._time_off = time_offset

    def __enter__(self):
        self._t_start = time.time()
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        self._t_end = time.time()

    def elapsed_time(self):
        return self._time_off + (self._t_end - self._t_start)

For more details about context managers, please refer elsewhere. Let's use our timer to time the kernel:

In [6]:
# Set up the host vectors
N = 200*1000*100
x = np.random.rand(N)
y = np.random.rand(N)

# Copy and allocate data on the device
d_x = cuda.to_device(x)
d_y = cuda.to_device(y)
d_z = cuda.device_array_like(x)

# Set up the kernel invocation
block_size = 128
num_blocks = N // block_size
if N % block_size:
    num_blocks += 1

# Call the kernel
with time_region() as t_kernel:
    _vecadd_cuda[num_blocks, block_size](d_z, d_x, d_y)

print(f'CUDA kernel time: {t_kernel.elapsed_time()} s')

# Copy back the result to the host
res = d_z.copy_to_host()

# Validate the result
assert np.allclose(x + y, res)

CUDA kernel time: 0.00018906593322753906 s


Our timer seems to work fine; we still measure the kernel launch time as with `%timeit`. In order to measure the actual kernel execution time, we need to block the CPU calling thread until the kernel finishes, immediately after we launch the kernel. We can achieve that with `cuda.synchronize()`:

In [7]:
# Set up the host vectors
N = 200*1000*1000
x = np.random.rand(N)
y = np.random.rand(N)

# Copy and allocate data on the device
d_x = cuda.to_device(x)
d_y = cuda.to_device(y)
d_z = cuda.device_array_like(x)

# Set up the kernel invocation
block_size = 128
num_blocks = N // block_size
if N % block_size:
    num_blocks += 1

# Call the kernel
with time_region() as t_kernel:
    _vecadd_cuda[num_blocks, block_size](d_z, d_x, d_y)
    cuda.synchronize()

with time_region() as t_ref:
    z = x + y

print(f'CUDA kernel time: {t_kernel.elapsed_time()} s')
print(f'Numpy time:       {t_ref.elapsed_time()} s')


# Copy back the result to the host
res = d_z.copy_to_host()

# Validate the result
assert np.allclose(x + y, res)

CUDA kernel time: 0.00887155532836914 s
Numpy time:       0.09877943992614746 s


Not bad, the CUDA kernel is 10x faster than the native Numpy kernel.

Before analysing how good or bad this is, let's see an alternative way for measuring the kernel time that actually avoids the use of `cuda.synchronize()`.

## Measuring the kernel execution time with CUDA events

Inserting `cuda.synchronize()` without a reason could slow down your application, since it not only blocks the current CPU thread, but also imposes a synchronization point for all the CUDA streams on the GPU that are currently running in parallel.

> A CUDA stream is essentially a series of sequential operations (data transfers, kernel launches, etc.) that execute on the GPU. Multiple CUDA streams may run independently on the GPU, thus allowing overlapping of operations, such as data transfers and execution of kernels.

To avoid this, but also to obtain a more precise measurement, you can use [CUDA events](http://docs.nvidia.com/cuda/cuda-c-programming-guide/#events). You can imagine CUDA events as milestones associated with timestamps that you can insert between operations in a CUDA stream. Let's how we can adapt our `time_region` context manager to use CUDA events:

In [1]:
class time_region_cuda:
    def __init__(self, time_offset=0, cuda_stream=0):
        self._t_start = cuda.event(timing=True)
        self._t_end = cuda.event(timing=True)
        self._time_off = time_offset
        self._cuda_stream = cuda_stream

    def __enter__(self):
        self._t_start.record(self._cuda_stream)
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        self._t_end.record(self._cuda_stream)
        self._t_end.synchronize()

    def elapsed_time(self):
        return self._time_off + 1.e-3*cuda.event_elapsed_time(self._t_start, self._t_end)

To measure a data region with CUDA events you first need to create two events: one for the start and one for the end. You can achieve that with the `cuda.event(timing=True)`. To start counting, you need to call `record()` on the starting event marking the "arrival" to that milestone. Similarly, you call `record()` on the ending event to mark the end of the region. Then you can obtain the elapsed time using the corresponding function as shown in the example above.

Let's rewrite our vector addition example using the CUDA event timers:

In [9]:
# Set up the host vectors
N = 200*1000*1000
x = np.random.rand(N)
y = np.random.rand(N)

# Copy and allocate data on the device
d_x = cuda.to_device(x)
d_y = cuda.to_device(y)
d_z = cuda.device_array_like(x)

# Set up the kernel invocation
block_size = 128
num_blocks = N // block_size
if N % block_size:
    num_blocks += 1

# Call the kernel
with time_region_cuda() as t_kernel:
    _vecadd_cuda[num_blocks, block_size](d_z, d_x, d_y)

with time_region() as t_ref:
    z = x + y

print(f'CUDA kernel time: {t_kernel.elapsed_time()} s')
print(f'Numpy time:       {t_ref.elapsed_time()} s')


# Copy back the result to the host
res = d_z.copy_to_host()

# Validate the result
assert np.allclose(x + y, res)

CUDA kernel time: 0.009249631881713868 s
Numpy time:       0.14605975151062012 s


As we can see the execution time obtained is the correct one without having to use `cuda.synchronize()`.

## Assessing the performance of the kernel

The question that arises is how good is the performance that we achieve. Let's inspect further the kernel. Each thread does two `float64` reads from the memory and one write and performs an addition. That means for one floating operation, the kernel must transfer to/from memory 24 bytes from the main memory. This gives us an  *arithmetic intensity* or *flop:byte ratio* of 0.0417. The lower this ratio is for a computational kernel, the more likely will be that the kernel is memory bandwidth bound. As the ratio increases, the kernel tends to be more compute bound. The theory behind the arithmetic intensity is covered by the *Roofline* performance model, which is outside the scope of this tutorial. For the moment, let's compute two performance metrics, the `Gflop/s` achieved by the kernel and the data transfer rate to/from memory:

In [12]:
print(f'Performance: {1.e-9*N/t_kernel.elapsed_time()} Gflop/s')
print(f'Transfer rate: {1.e-9*3*N*8/t_kernel.elapsed_time()} GB/s')

Performance: 21.622482122276843 Gflop/s
Transfer rate: 518.9395709346443 GB/s


Let's compute the same for the NumPy kernel:

In [14]:
print(f'Performance: {1.e-9*N/t_ref.elapsed_time()} Gflop/s')
print(f'Transfer rate: {1.e-9*3*N*8/t_ref.elapsed_time()} GB/s')

Performance: 1.3693026171241833 Gflop/s
Transfer rate: 32.8632628109804 GB/s


As you can, the GPU can deliver more than 10x bandwidth compared to the CPU. Checking the [datasheet](https://www.nvidia.com/content/dam/en-zz/Solutions/Data-Center/tesla-p100/pdf/nvidia-tesla-p100-datasheet.pdf) of the NVIDIA P100 GPU, we can see that the peak nominal memory bandwidth is 732 GB/s, meaning that our kernel utilizes 70% of the peak bandwidth.

> Achieving the nominal peak memory bandwidth is usually not possible with real-life computational kernels, even with very low arithmetic intensity. For this reason, we tend to cite the *effective memory bandwidth*, which is obtained by benchmarks like the one presented in this tutorial. In fact, the effective memory bandwidth of the P100 GPUs is at ~550 GB/s, which essentially shows that the vector addition kernel's performance is optimal. For the Haswell CPUs on the host, the effective memory bandwidth is ~50 GB/s.

## Understanding the data transfer overhead

So far we have only focused on the performance of the kernel. There is still a quite important topic we have not yet addressed. CUDA kernels require that the data they operate on is located on the device and we need to move that data there from the host. What is the cost of this data movement? Let's time our benchmark code:

In [8]:
# Set up the host vectors
N = 200*1000*1000
x = np.random.rand(N)
y = np.random.rand(N)

# Copy and allocate data on the device
with time_region_cuda() as t_copyin:
    d_x = cuda.to_device(x)
    d_y = cuda.to_device(y)

with time_region_cuda() as t_create:
    d_z = cuda.device_array_like(x)

# Set up the kernel invocation
block_size = 128
num_blocks = N // block_size
if N % block_size:
    num_blocks += 1

# Call the kernel
with time_region_cuda() as t_kernel:
    _vecadd_cuda[num_blocks, block_size](d_z, d_x, d_y)

with time_region() as t_ref:
    z = x + y

print(f'CUDA kernel time: {t_kernel.elapsed_time()} s')
print(f'Numpy time:       {t_ref.elapsed_time()} s')


# Copy back the result to the host
with time_region_cuda() as t_copyout:
    res = d_z.copy_to_host()

print(f'Copyin time:  {t_copyin.elapsed_time()}')  
print(f'Create time:  {t_create.elapsed_time()}')    
print(f'Copyout time: {t_copyout.elapsed_time()}')    


# Validate the result
assert np.allclose(x + y, res)

CUDA kernel time: 0.00925113582611084 s
Numpy time:       0.1446390151977539 s
Copyin time:  0.39932965087890626
Create time:  0.020876127243041994
Copyout time: 0.7593539428710938


The data copy times are quite important! In fact, if we include these in the total execution time of the GPU version, the CPU version becomes more than 8x faster! Data transfers is the No. 1 optimization that you should do when programming for the GPUs. You must minimize the data transfers to/from GPU by keeping the necessary data on the GPU for as long as it is needed.

Before closing this discussion, let's see how fast is the data moved over to the GPU:

In [14]:
print(f'Copyin rate: {1e-9*2*N*8/t_copyin.elapsed_time()} GB/s')

Copyin rate: 8.013429488536469 GB/s


This is bound by the data rate of the PCI 16x bus where the GPU is attached to and, it is indeed way too slower than the main memory bandwidth of modern processors.

Interestingly, the copyout data rate seems to be much slower:

In [21]:
print(f'Copyout rate: {1e-9*N*8/t_copyout.elapsed_time()} GB/s')

Copyout rate: 2.107054312446775 GB/s


By default, memory allocated on the host is pageable. This means that it can be moved by the OS kernel to a secondary storage device if there is not enough memory available on the system. This can incur a significant performance penalty, especially if you write on freshly allocated memory (as it happens in our example). You can avoid this overhead by using *page-locked* or *pinned* memory. This memory cannot be paged out and it is physically resident on the memory device. CUDA gives you the opportunity to use pinned memory and Numba allows you to create pinned ndarrays using the [cuda.pinned_array()](http://numba.pydata.org/numba-doc/latest/cuda-reference/memory.html#numba.cuda.pinned_array) function.

> In order to keep track of which memory pages are resident on the physical memory and which are not, the OS kernel maintains a special data structure called *page table*. When you allocate memory on the host, the OS kernel simply creates a virtual memory mapping and it does not allocate any physical page. As soon as you start writing to the memory area you have been allocated, it will look for the page in its page tables and if not found, a *page fault* will be raised and then the kernel will have to physically allocate the missing memory page and update its page tables.

Let's rewrite the copyout part using pinned memory:

In [6]:
res = cuda.pinned_array(N)
with time_region_cuda() as t_pinned:
    d_z.copy_to_host(res)
    
assert np.allclose(x + y, res)
print(f'Copyout data rate (pinned): {1e-9*N*8/t_pinned.elapsed_time()} GB/s')

NameError: name 'cuda' is not defined

Notice how much the performance has improved. It is now even better than the copyin operation.

However, pinned memory does not come without a cost. Since pinned pages cannot be paged out, they will stay on the physical memory, increasing the memory pressure and, finally, the effective memory consumption of the code. For memory hungry applications, this can be a problem.

### Exercise

> Apply the pinned memory technique also to the input arrays `x` and `y`.

## Profiling the CUDA code

In this simple example of vector addition we assessed the performance and identified the bottlenecks ourselves, by analyzing the code structure and reasoning about it. In more complex codes or codes that you are not very familiar with, it would be good if this analysis could be done by a dedicated tool. Not to be misunderstood, understanding the code structure and its memory and compute requirements is essential for optimizing it in any case, but using a *performance profiler* is very handy for analyzing the performance bottlenecks, for helping you prioritizing your optimization targets and for understanding how much room for improvement exists.

NVIDIA provides [Nsight](https://developer.nvidia.com/nsight-visual-studio-edition) and `nvprof` for profiling CUDA code. In this tutorial, we are going to use `nvprof` and the [Nvidia Visual Profiler](https://developer.nvidia.com/nvidia-visual-profiler) to inspect the results.

> You may install the Nvidia Visual Profiler on your personal computer and visualize the performance results, even if you don't have a GPU.

The `src/vecadd.py` file contains the vector addition example as we have finally presented it here. Let's do a basic profing:

```bash
srun -C gpu nvprof -o vecadd.nvprof python src/vecadd.py $((200*1000*1000))
```

> If you want to profile your code with `nvprof` you should call `cuda.profile_stop()` at the end of your program.

This profiling provides basic information about the data transfers and the execution time of the different kernels. It is the first step you need to take, because this will show you how much time you spend on transferring data to/from the device and which kernels are the most time consuming. Here is a screenshot for our example:

![Profiling of the vector addition benchmark](figs/vecadd-nvprof.png)

Notice how much is the overhead of the data transfers as well as that of the pinned memory allocation of the `res` array. Placing your cursor on top of any of the regions in the timeline you can see more information. In this case, I have highlighted the copy-to-host operation, where you can see that the target memory on the host is pinned and the data rate is exactly as the one we have calculated above.

As soon as you have addressed the data transfer issues, the next step is to identify the performance bottlenecks in the most time consuming kernels. To do that, you need more detailed information that reflects hardware events happening on the GPU (e.g., instructions executed, data transferred from the main memory, use of caches etc.). To achieve this you have to pass the `--analysis-metrics` option to `nvprof` as follows:

```bash
srun -C gpu nvprof -o vecadd.detailed.nvprof --analysis-metrics python src/vecadd.py $((200*1000*1000))
```

As you will notice this command incurs quite of an overhead. The reason behind that is that there are only a few hardware performance monitoring registers on the GPU, so in order for `nvprof` to collect all the necessary performance metrics, it will have to rerun the kernel several times.

From this type of analysis you can obtain the actual memory bandwidth consumed by your kernel and this is what shows up for our vector addition kernel:

![Detailed profiling of the vector addition benchmark](figs/vecadd-detailed-nvprof.png)

As expected the memory bandwidth consumption is red highlighted since it is the performance limiting factor, as we have also calculated manually. The bandwidth consumption reported here is 557 GB/s, which is quite close with our measurements based on the algorithm details.

This concludes our discussion on measuring and analyzing the performance of a CUDA program written with Numba.